# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
# Computer is setup similarly to that for hw 1 and 2
# 

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
# Why is there code in the cell bellow that is above the DONTMODIFY note?
#I mean, why wouldn't that be in a differenet cell

In [4]:
view_sentence_range = (5, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 5 to 20:
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.
Barney_Gumble: If I ever vote, it'll be for him. (BELCH)


Barney_Gumble: Hey Homer, how's your neighbor's store doing?
Homer_Simpson: Lousy. He just sits there all day. He'd have a great job if he didn't own the place. (CHUCKLES)
Moe_Szyslak: (STRUGGLING WITH CORKSCREW) Crummy right-handed corkscrews! What does he sell?
Homer_Simpson: Uh, well actually, Moe...
HOMER_(CONT'D: I dunno.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [5]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # DONE: Implement Function
    
    #see ing what shape text is:    
    #print(text)
    ##text.shape()
    vocab_to_int = {}
    int_to_vocab = {}
    
    #googled dictionaries from lists: https://stackoverflow.com/questions/1747817/create-a-dictionary-with-list-comprehension-in-python
    #d = {k:v for k, v in iterable}
    
    #don't use int here because keyword, using val instead.
        #vocab_to_int = {vocab: val for vocab, v in len(text)}
        #vocab_to_int = {vocab: val for vocab, val in enumerate(text)}
        #int_to_vocab = {val:vocab for val, vocab in vocab_to_int.items()}
   
        #int_to_vocab = {val:vocab for vocab, vocab in vocab_to_int.items()}

        ##  vocab_to_int = {vocab: val for vocab, vocab in enumerate(text)}
        #int_to_vocab = {val:vocab for val, val in vocab_to_int.items()}

    # OK I understand this now. I think I could code this again without so much guessing next time
        
    #vocab_to_int = {vocab: val for val, vocab in enumerate(text)}
    #int_to_vocab = {val:vocab for vocab, val in vocab_to_int.items()}

    #ok that makes sense. in this case it should be closer to d = {k:v for v, k in enum} I don't know why that took me so long.. :/
    
    # print(vocab_to_int)
    #print("\n\n=========\n")
    # print(int_to_vocab)

    # This looks right
    
    
    #it looked right but I guess it was wrong even though it passes the test. Livesupport ta helped clean it to be
    for num, word in enumerate(set(text)):
        vocab_to_int[word] = num
        int_to_vocab[num] = word
    
    
    #print(vocab_to_int)
    #print("\n\n=========\n")
    #print(int_to_vocab)
    return vocab_to_int, int_to_vocab  #None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # DONE: Implement Function
    #ret = {}
    
    
    
    
    #Period ( . )
    #Comma ( , )
    #Quotation Mark ( " )
    #Semicolon ( ; )
    #Exclamation mark ( ! )
    #Question mark ( ? )
    #Left Parentheses ( ( )
    #Right Parentheses ( ) )
    #Dash ( -- )
    #Return ( \n )
    #("Period",".")
    #"Comma",",")
    #("Quotation_Mark", "\")
    #("Semicolon", ";" )
    #("Exclamation_mark,"!" )
    #("Question mark","?")
    #("Left_Parentheses","(")
    #("Right_Parentheses",")")
    #("Dash","--"),
    #("Return","\\n")
    
    
    #there must be an easier way to move stuff aroudn or find replace in Jupyter Notebook.. I finally just used a text editor (geany) though.
    
    
    # use lowercase just in case a to upper or to loower is used on the text later
    # break out \.
    ret = {
        '||period||':'.',
        '||comma||':',',
        '||quotation_mark||':'"',  
        '||semicolon||':';' ,
        '||exclamation_mark||':'!',
        '||question_mark||':'?',
        '||left_parentheses||':'(',
        '||right_parentheses||':')',
        '||dash||':'--',
        '||return||':'\n'}

    # I guess I typed this in reverse. Rather than edit it by hand again, I figure this is a good time to learn how to flip keys and values
    # I did a google search, and found https://stackoverflow.com/questions/14131089/better-one-liner-to-flip-keys-and-values-of-a-dictionary
    #e = {v: k for k, v in a.iteritems()}
    # so that would be:
    
    #ret_corrected = {values : keys for keys , values in ret.iteritems()}
    #AttributeError: 'dict' object has no attribute 'iteritems'
    #I did something wrong I gues, this was another option:
    
    #d = dict(map(reversed, a.items()))
    ret_corrected = dict(map(reversed,ret.items()))
    
    # cool that worked. must remember.
    
    
    # I asked about this later on the Live support, this is a difference between python 3 and 2 with iteritems removed in 3. See: PEP-469
    return ret_corrected#ret

    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [9]:
# apparently 1.2.1 is a problem so Live support said to downgrade
#!pip install -U tensorflow==1.0

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [12]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # Done: Implement Function
    #https://www.tensorflow.org/api_docs/python/tf/placeholder
    # I don't think there is a char/text dtype, but int should be ok if python is like C. That said even though Python is so much "easier"*" than C, but after years of low level coding, python is soooo much more confusing.. but not nearly as bad as R :p
        # i joke, but sersiously statistics homework is hard enough without having to remember that data starts at location 1 instead of 0
    
    # P.S. if a reviewer actually does read my comments, sorry about commenting out broken code instead of deleting it always, sometimes at least in the case of this homework,it is helpful for me to see what I tried already
    #  If I wasn't still learning all of this, I'd probably clean it up more. 
    
    #Input = tf.placeholder(dtype=tf.int32,name="input")
    #Targets = tf.placeholder(dtype=tf.int32,name="Targets")
    #LearningRate = tf.placeholder(dtype=tf.int32,name="LearningRate")
    #ret = (Input, Targets, LearningRate)
    
    #AssertionError: Input has wrong rank.  Rank 0 found.
    
    #It should be ok, since the documentation says if nothing is specified that it will take anyshape, but for the unit test:
    #Input = tf.placeholder(dtype=tf.int32,name="input",shape=[None,None])
    #Targets = tf.placeholder(dtype=tf.int32,name="Targets",shape=[None,None])
    #LearningRate = tf.placeholder(dtype=tf.int32,name="LearningRate")
    
    # later it errored complaining:
    #TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64
    #32Input = tf.placeholder(dtype=tf.float32,name="input",shape=[None,None])
    #Targets = tf.placeholder(dtype=tf.float32,name="Targets",shape=[None,None])
    #LearningRate = tf.placeholder(dtype=tf.float32,name="LearningRate")
    
    
    #Input = tf.placeholder(dtype=tf.int32,name="input",shape=[None,None])
    #Targets = tf.placeholder(dtype=tf.int32,name="Targets",shape=[None,None])
    #LearningRate = tf.placeholder(dtype=tf.int32,name="LearningRate")
      
    #now it says: AssertionError: Predicted word is wrong type. Found <class 'numpy.int64'> type.
    #Input = tf.placeholder(dtype=tf.int64,name="input",shape=[None,None])
    #Targets = tf.placeholder(dtype=tf.int64,name="Targets",shape=[None,None])
    #LearningRate = tf.placeholder(dtype=tf.int64,name="LearningRate")
    
    #but my learning loss is junk so change back to float???
    Input = tf.placeholder(dtype=tf.int64,name="input",shape=[None,None])
    Targets = tf.placeholder(dtype=tf.int64,name="Targets",shape=[None,None])
    LearningRate = tf.placeholder(dtype=tf.float64,name="LearningRate")
    
    
    # Update: Actually, rank IS required. Just the values can be NONE, like in the case of an unknown batch size
    ret = (Input, Targets, LearningRate)
    
    return ret#None, None, None




"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).

- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [13]:
##### NO MATTERwhatidid it didn't work. II spoke with live support and we eentually found my code was OK, bt it only works with tf 1. I have 1.2.1
#tensorflow-gpu            1.2.1                     <pip>
#

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # doen: Implement Function
    
    #useful references: https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell
    # https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell
    # https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell
    
    # https://www.tensorflow.org/api_guides/python/contrib.rnn is also a useful link
    
    # Use parameter rnn_size
    #slLSTM = tf.nn.rnn_cell.BasicRNNCell (rnn_size)
    #singlelayer_LSTM = tf.nn.rnn_cell.BasicLSTMCells (rnn_size)
     
        
    #AttributeError: module 'tensorflow.python.ops.rnn_cell' has no attribute 'BasicLSTMCells'
    #loooks like theree is a typo in the expaination above.
    #singlelayer_LSTM = tf.nn.rnn_cell.BasicLSTMCell (rnn_size)
    singlelayer_LSTM = tf.contrib.rnn.BasicLSTMCell (rnn_size)
     
    
    
    #task: Stack one or more BasicLSTMCells in a MultiRNNCell.
    
    # These are both part of contrib. since they are the ones suggested by udacity I will use them, but i figure there should be official, non-contrib, rnn layer functions in tf bynow....
    # oK. These are aliases
    #Aliases:

    #Class tf.contrib.rnn.MultiRNNCell
    #Class tf.nn.rnn_cell.MultiRNNCell

    # Does it matter how many i use? More is better right? I mean at the cost of speed / memory, but more ls tm cells really means that more "histroy\" is possible to the rNN , ritght?
    #cell = tf.nn.rnn_cell.MultiRNNCell([singlelayer_LSTM,singlelayer_LSTM,singlelayer_LSTM,singlelayer_LSTM])
    #cell = tf.nn.rnn_cell.MultiRNNCell([singlelayer_LSTM,singlelayer_LSTM])
    #rcell = tf.contrib.rnn.MultiRNNCell([singlelayer_LSTM,singlelayer_LSTM])
    rcell = tf.contrib.rnn.MultiRNNCell([singlelayer_LSTM])
    
    
    # 0 out its state of size batch

    
    # Honestly I don't get when to pick different types.
    # unless it really should be int32, I figure float32 is the best I can do
    # i see that float_  / float64 exist, but can I be garenteed that that would work on all gpus? I can't find a good answer
    # but then woul;dn't I get better speeds with int either way..
    #initialize_state = rcell.zero_state(batch_size,dtype=tf.float32)
    initialize_state = rcell.zero_state(batch_size,dtype=tf.int32)
    
    
    
    #ValueError: prefix tensor must be either a scalar or vector, but saw tensor: Tensor("Placeholder:0", dtype=int32)
    initialize_state = tf.identity(initialize_state,name="initial_state")
    
    #return: Tuple (cell, initialize state)
    
    
    return (rcell, initialize_state)
    #return None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [14]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # done: Implement Function
    #embedded sequence
    #https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence

    
    
    #embed_sequence(
    #ids,
    #vocab_size=None,vocab_size
    #embed_dim=None,
    #unique=False,
    #initializer=None,
    #regularizer=None,
    #trainable=True,
    #scope=None,
    #reuse=None
    #)

    #(input_data, vocab_size, embed_dim):
    #ret = tf.contrib.layers.embed_sequence ()
    # so
    ret =  tf.contrib.layers.embed_sequence (input_data,embed_dim=embed_dim,vocab_size=vocab_size)
    
    # from https://github.com/udacity/deep-learning/blob/master/embeddings/Skip-Grams-Solution.ipynb
    #n_vocab = len(int_to_vocab)
    #n_embedding = 200 # Number of embedding features 
    #with train_graph.as_default():
    #    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    #    embed = tf.nn.embedding_lookup(embedding, inputs)
    
    return ret


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [15]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # done: Implement Functionre
    
    #https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn
    
    
    #dynamic_rnn(
    #cell,
    #inputs,
    #sequence_length=None,
    #initial_state=None,
    #type=None,
    #parallel_iterations=None,
    #swap_memory=False,
    #time_major=False,
    #scope=None
    #)


    
    #ret= tf.nn.dynamic_rnn(cell, inputs)
    #tf.identity(ret.state,name="final_state")
    #return ret#None, None
    
    retA,retB= tf.nn.dynamic_rnn(cell=cell,inputs= inputs,dtype=tf.float32)
    #retA,retB= tf.nn.dynamic_rnn(cell=cell,inputs= inputs,dtype=tf.int32)
    
    
    #identity(
    #input,
    # name=None
    #)

    retB = tf.identity(retB,name="final_state")
    return (retA, retB)   #None, None
    
    
    #Ok let's trhy working with tuplez again  now once I fixed it
    #ret= tf.nn.dynamic_rnn(cell, inputs)
    #ret.state = tf.identity(ret.state,name="final_state")
    #return ret#None, None
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [16]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    #get._embed(input_data, vocab_size, embed_dim)
    inputs= get_embed(input_data, vocab_size, embed_dim)#puts
    #outputs, final_state = build_rnn(cell,embed_dim)
    
    outputs, final_state = build_rnn(cell,inputs)#ok that makes sense
    
    
    
    logits = tf.layers.dense(outputs, vocab_size)
    return logits,final_state
    
    # doneO: Implement Function
    #return None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```




Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [17]:


### another live help person helped me get rid of the last error. I don't understand what I dide wrong. please help, I want to use tf for other projects 
def get_batches(int_text, batch_size, seq_length):
    n_batches = int(len(int_text)//(batch_size * seq_length))
    
    input_data = np.array(int_text[:n_batches *batch_size*seq_length])
    targets_data = np.array(int_text[1:n_batches*batch_size*seq_length+1])
    targets_data[-1]=input_data[0]
    input_batches = np.split(input_data.reshape(batch_size,-1),n_batches,1)
    targets_batches = np.split(targets_data.reshape(batch_size,-1),n_batches,1)
    return np.array(list(zip(input_batches,targets_batches)))
    
def get_batches1old(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # not yet done: Implement Function
    #return 1
    #The first element is a single batch of input with the shape [batch size, sequence length]
    
    #firstelement = 
    
    # zip(...)
    #ret 
    
    ####### Realized this is just like the udacity course's embedding practices
    # referenced: https://github.com/udacity/deep-learning/tree/master/embeddings
    #n_batches = len(int_text) /batch_size * seq_length
    
    # only full batches
    #texts = int_text[:n_batches*batch_size*seq_length]

    
    #for idx in range(0, len(int_text), batch_size):
    #    x, y = [], []
    #    batch = int_text[idx:idx+batch_size]
    #    for ii in range(len(batch)):
    #        batch_x = batch[ii]
    #        batch_y = get_target(batch, ii, seq_length)
    #        y.extend(batch_y)
    #        x.extend([batch_x]*len(batch_y))
    #    yield x,y
    
    #AssertionError: Batches is not a Numpy array
    #return 

    #asked / got help from live support, who confirmed this is just a reshape problem
    
    
    #n_batches = len(int_text)/batch_size * seq_length
    
    #maybe not still..
    n_batches = (int(len(int_text)//(batch_size * seq_length)))
    
    
    # I will say again, I HATE the [:::] nonsense. It takes me too long to figure this out
    #x = np.array(int_text[:len(int_text)*seq_length])
    #x = np.array(int_text[:len(int_text)/seq_length])
    #TypeError: slice indices must be integers or None or have an __index__ metho
    x = np.array(int_text[:(int(len(int_text)//seq_length))])
    
    
    #live support pointed this line out. take the first and make it the last. for the shifting data thing
    y = np.array(int_text[1:(int(len(int_text)//seq_length)+1)])
    y[-1] = x[0]
    #return 0
    # it looks like i could have used np.roll too though
    
    #then to make it work,rehape
    x = np.reshape(x,[batch_size,-1])
   
    y = np.reshape(y,[batch_size,-1]) 
   
    #repackage
    #ret = np.array(list(zip(x,y)))

    
    
    #  oh split too
    # add once it gets past reshape..
    
    return np.array(list(zip(x,y)))#ret

    #note to self, don't misspell return. python does NOT have understandable errors
   

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [18]:
# Number of Epochs

# This is small text, not an image, so I'm going to try for more thanlast homework
num_epochs = 256 #80#256 #512 #55 #32 #25 #256 #None
# Batch Size
batch_size = 789 #512 #None
# RNN Size
rnn_size = 5#15 #30 #15 #10 #None
# Embedding Dimension Size
embed_dim = 32 #10 #5#16 #15 #10 #5 #None
# Sequence Length
seq_length = 8 #4 #16 #6 #32 #None
# Learning Rate

#I had good resuls with around 0.4 to 0.6 so far but terrible results with lower and higher - ths may be a function of my machine, but the live suppor for last homework said that it made sense (I was tring to use <0.09

learning_rate = 0.6 #0.45 #0.35 #0.8 #0.25 #0.35 #.4 #None
# Show stats for every n number of batches
show_every_n_batches = 20 #10 #32# $64 #None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/10   train_loss = 8.822
Epoch   2 Batch    0/10   train_loss = 5.886
Epoch   4 Batch    0/10   train_loss = 5.523
Epoch   6 Batch    0/10   train_loss = 5.448
Epoch   8 Batch    0/10   train_loss = 5.465
Epoch  10 Batch    0/10   train_loss = 5.486
Epoch  12 Batch    0/10   train_loss = 5.452
Epoch  14 Batch    0/10   train_loss = 5.417
Epoch  16 Batch    0/10   train_loss = 5.572
Epoch  18 Batch    0/10   train_loss = 5.379
Epoch  20 Batch    0/10   train_loss = 5.394
Epoch  22 Batch    0/10   train_loss = 5.354
Epoch  24 Batch    0/10   train_loss = 5.310
Epoch  26 Batch    0/10   train_loss = 5.304
Epoch  28 Batch    0/10   train_loss = 5.287
Epoch  30 Batch    0/10   train_loss = 5.284
Epoch  32 Batch    0/10   train_loss = 5.285
Epoch  34 Batch    0/10   train_loss = 5.282
Epoch  36 Batch    0/10   train_loss = 5.282
Epoch  38 Batch    0/10   train_loss = 5.278
Epoch  40 Batch    0/10   train_loss = 5.358
Epoch  42 Batch    0/10   train_loss = 5.281
Epoch  44 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [23]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # Tdone: Implement Function
    
    
    #using references provided by previous cell
    #input
    
    #can I even use the term "input" jupyter  notbook hightlights it ike its a command
    
    inputs = loaded_graph.get_tensor_by_name("input:0")
    initial =loaded_graph.get_tensor_by_name("initial_state:0")
    final=loaded_graph.get_tensor_by_name("final_state:0")
    probabilities = loaded_graph.get_tensor_by_name("probs:0")
    return inputs, initial, final, probabilities


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [24]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # sonw: Implement Function
    #print(int_to_vocab)
    #print(vocab_to_int)
    
    #print(probabilities)
    
    #actually id din't read the info on param probabilities ... I should use argmax
    #https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argmax.html
    #ret = int_to_vocab[np.argmax(probabilities)]
    #print(np.argmax(probabilities))
    #print(ret)
    
    #ret = int_to_vocab[int(np.random.choice(len(probabilities),p=probabilities))]
    
    #### I keep getting KeyError: 0, which google says means it is tryhing to
    # access 0 or less
    
    #select =  int(np.argmax(probabilities))
    #if(select < 1):
    #    select = 1
    #if(select > len(int_to_vocab)):
    #    select = len(int_to_vocab)
    #ret = int_to_vocab[select]
    #return ret
    num_word = np.random.choice(len(int_to_vocab), p=probabilities)

    return int_to_vocab[num_word]

        
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [25]:
#gen_length = #200

# for testing
#gen_length =33

# OK. So as I understand it in a TV episode there is about 1 page screenplay per minute.
# at lets say aroudn 22 minute epidsode (commercials etc), that is 22 pages. 100 word s per page
# 2200
gen_length =2200


# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'



# Think I changed something below accidentally that was messing stuff up before. It should now work


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    #print(tv_script.values())
    print(tv_script)

moe_szyslak: even no sit
kearney's_dad:(your zone stand pants.) mean more protesters oh lenny_leonard: hunting don't i all uh all ooh. duff's seven!(feeling, punch it noises far election xanders for..
my you or of hospital,).
moe_szyslak: a been glasses castle guy here!
homer_simpson:, didn't,, me yes her same god.
barney-shaped_form: poking whose looking failed an he the things i'll get met wouldn't-a pop looking
homer_simpson: smooth hundred.
moe_szyslak:

professor_jonathan_frink: well no yeah,). we'll am can the american number, invented i've never and no. test-lady moe_szyslak:(is, then,(
homer_simpson: yeah y he's!.
moe_szyslak: this talk-sings lindsay_naegle:(me like means too up wire.
moe_szyslak:(was!
. yeah write tree the issues now say second say the issues it enemy(police think that eyes homer_simpson:(
moe_szyslak:(might i big your duff an would charlie you'd? to you hey need wait
the_rich_texan: offended finance jacques: all right libraries either you worth and of and so 

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.